<a href="https://colab.research.google.com/github/Kridr/HW_PB_HSE/blob/main/Project_PB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.1 MB 6.3 MB/s 


In [ ]:
from Bio import SeqIO
from Bio import Entrez
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [ ]:
# Create a sequence
with open("fragment22.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"): # Там всего одна последовательность
        sequence_object = Seq(record.seq)

In [ ]:
import pandas as pd

In [ ]:
names = [
    'query acc.ver', 
    'subject acc.ver', 
    '% identity', 
    'alignment length', 
    'mismatches', 
    'gap opens', 
    'q. start', 
    'q. end', 
    's. start', 
    's. end', 
    'evalue', 
    'bit score', 
    '% positives'
]

uniprot_df = pd.read_csv('uniprot_blastp.csv', header=None, names=names)
pdb_df = pd.read_csv('pdb_blastp.csv', header=None, names=names)
refseq_select_prot_df = pd.read_csv('refseq_select_prot_blastp.csv', header=None, names=names)
pataa_df = pd.read_csv('pataa_blastp.csv', header=None, names=names)
refseq_protein_df = pd.read_csv('refseq_protein_blastp.csv', header=None, names=names)
tsa_nr_df = pd.read_csv('tsa_nr_blastp.csv', header=None, names=names)
env_nr_df = pd.read_csv('env_nr_blastp.csv', header=None, names=names)

uniprot_df.head()

,query acc.ver,subject acc.ver,% identity,alignment length,mismatches,gap opens,q. start,q. end,s. start,s. end,evalue,bit score,% positives
0,lcl|ORF49:31045:31941,Q966L9.1,44.144,111,50,3,170,276,69,171,3.290000e-05,48.9,48.65
1,lcl|ORF84:812:1537,P75853.2,38.889,198,113,4,47,241,27,219,4.740000e-42,148.0,60.10
2,lcl|ORF84:812:1537,P40400.1,29.208,202,133,5,36,231,20,217,3.150000e-17,82.4,50.50
3,lcl|ORF99:41885:42196,P42063.1,44.118,34,19,0,3,36,29,62,2.000000e-02,36.2,61.76
4,lcl|ORF100:42377:42619,P24139.1,42.373,59,34,0,2,60,108,166,2.890000e-05,43.1,62.71


In [ ]:
orfs = {}
with open('__all.fa') as f:
    seq = ''
    name = ''
    for line in f.readlines():
        if line and line[0] == '>':
            if seq and name:
                orfs[name] = seq
            seq = ''
            name = line.split()[0][1:]
        else:
            seq += line.strip()

In [ ]:
# Create a record
record = SeqRecord(sequence_object,
                   id='fragement22', # random accession number
                   name='fragement22',
                   description='fragement22')
record.annotations = {'molecule_type': 'DNA'}

In [ ]:
number = 0

In [ ]:
s = set()
def add_features_from_df(record, number, s, df, identity_th=99, evalue_th = 1e-3):
    #global record, number
    for i in df.index:
        if df.at[i, '% identity'] > identity_th and df.at[i, 'evalue'] < evalue_th:
            if df.at[i, 'query acc.ver'] in s:
                continue
            try:
                e = Entrez.efetch(db="biosample", id=df.at[i, 'subject acc.ver'], retmode='text')
                e = e.read().split('\n')

                locations = df.at[i, 'query acc.ver'].split(':')
                loc1 = int(locations[1])
                loc2 = int(locations[2])

                start = min(loc1, loc2)
                end = max(loc1, loc2)
                strand = 1 if loc1 < loc2 else -1
                feature = SeqFeature(
                    FeatureLocation(start=start, end=end, strand=strand),                
                    qualifiers={
                        'gene': e[0],
                        'product': df.at[i, 'subject acc.ver'],
                        'translation': orfs[df.at[i, 'query acc.ver']][df.at[i, 'q. start']:df.at[i, 'q. end']],
                        'note': f'{e[1]} # Identity: {df.at[i, "subject acc.ver"]} # Positives: {df.at[i, "% positives"]} # Evalue: {df.at[i, "evalue"]}',
                    }, 
                    type='gene'
                )
                record.features.append(feature)
                number += 1
                s.add(df.at[i, 'query acc.ver'])
            except Exception as r: 
                print(r)

In [ ]:
add_features_from_df(record, number, s, uniprot_df, identity_th=90)
add_features_from_df(record, number, s, pdb_df, identity_th=90)
add_features_from_df(record, number, s, refseq_select_prot_df, identity_th=90)
add_features_from_df(record, number, s, pataa_df, identity_th=90)
add_features_from_df(record, number, s, refseq_protein_df, identity_th=90)
add_features_from_df(record, number, s, tsa_nr_df, identity_th=90)
add_features_from_df(record, number, s, env_nr_df, identity_th=90)

/usr/local/lib/python3.8/dist-packages/Bio/Entrez/__init__.py:686: UserWarning: 
            Email address is not specified.

            To make use of NCBI's E-utilities, NCBI requires you to specify your
            email address with each request.  As an example, if your email address
            is A.N.Other@example.com, you can specify it as follows:
               from Bio import Entrez
               Entrez.email = 'A.N.Other@example.com'
            In case of excessive usage of the E-utilities, NCBI will attempt to contact
            a user at the email address provided before blocking access to the
            E-utilities.
  warnings.warn(


In [ ]:
# Save as GenBank file
with open('my.gb', 'w') as output_file:
    SeqIO.write(record, output_file, 'genbank')

In [ ]:
s

{'lcl|ORF146:40929:41111',
 'lcl|ORF160:43946:43869',
 'lcl|ORF161:43859:43776',
 'lcl|ORF169:38333:29508',
 'lcl|ORF207:31459:31280',
 'lcl|ORF248:14551:14453',
 'lcl|ORF256:8380:8195',
 'lcl|ORF257:8122:8018',
 'lcl|ORF275:39729:38983',
 'lcl|ORF49:31045:31941'}

In [ ]:
len(s)

10

In [72]:
with open('__all_1.fa', 'w') as igfa:
    for k, v in orfs.items():
        if k not in s:
            igfa.write(f'>{k}\n')
            subseq = v
            subseqs70 = [subseq[i:i + 70] for i in range(0, len(subseq), 70)]
            for subseq70 in subseqs70:
                igfa.write(subseq70)
                igfa.write('\n')

Межгенные участки

In [ ]:
locs = {'start': [], 'end': []}

s = set()

for e in orfs.keys():
    locations = e.split(':')
    loc1 = int(locations[1])
    loc2 = int(locations[2])

    start = min(loc1, loc2)
    end = max(loc1, loc2)

    for i in range(start, end+1):
        s.add(i)

    locs['start'].append(start)
    locs['end'].append(end)

locs_df = pd.DataFrame.from_dict(locs, orient='columns')
locs_df.sort_values('start', ascending=True, inplace=True, ignore_index=True)

intergene = set([i for i in range(1, len(record.seq)+1)]) - s

In [ ]:
intergene = sorted(list(intergene))
intergene_se = {'start': [], 'end': []}
start = intergene[0]
end = intergene[0]

for i in range(1, len(intergene)):
    if intergene[i] - end > 1:
        intergene_se['start'].append(start)
        intergene_se['end'].append(end)

        start = intergene[i]
        end = intergene[i]
    else:
        end = intergene[i]

intergene_df = pd.DataFrame.from_dict(intergene_se, orient='columns')

In [ ]:
intergene_df

,start,end
0,103,439
1,1580,1627
2,1917,2441
3,2863,3015
4,3184,3295
...,...,...
84,48355,48423
85,48541,48678
86,48811,48816
87,48967,49266


In [ ]:
with open('intergene.fa', 'w') as igfa:
    for i in intergene_df.index:
        igfa.write(f'>IG{i}:{intergene_df.at[i, "start"]}:{intergene_df.at[i, "end"]}\n')
        subseq = str(record.seq)[intergene_df.at[i, "start"]+1:intergene_df.at[i, "end"]+2]
        subseqs70 = [subseq[i:i + 70] for i in range(0, len(subseq), 70)]
        for subseq70 in subseqs70:
            igfa.write(subseq70)
            igfa.write('\n')

BLASTN

In [ ]:
blastn_df = pd.read_csv('blastn.csv', header=None, names=names)
blastn_df.head()

,query acc.ver,subject acc.ver,% identity,alignment length,mismatches,gap opens,q. start,q. end,s. start,s. end,evalue,bit score,% positives
0,IG0:103:439,CP047242.1,91.691,337,28,0,1,337,104306,104642,3.080000e-127,468.0,NaN
1,IG0:103:439,CP034058.1,91.691,337,28,0,1,337,3660846,3661182,3.080000e-127,468.0,NaN
2,IG0:103:439,CP000117.1,91.691,337,28,0,1,337,5169517,5169853,3.080000e-127,468.0,NaN
3,IG0:103:439,AP025732.1,83.784,333,54,0,5,337,5578351,5578019,1.180000e-81,316.0,NaN
4,IG0:103:439,CP024785.1,82.143,336,60,0,2,337,7411609,7411944,2.570000e-73,289.0,NaN


In [ ]:
blastn_df[(blastn_df['% identity'] > 95) & (blastn_df['evalue'] < 1e-3)]

,query acc.ver,subject acc.ver,% identity,alignment length,mismatches,gap opens,q. start,q. end,s. start,s. end,evalue,bit score,% positives
44,IG5:4344:4424,CP047242.1,95.062,81,4,0,1,81,108547,108627,9.470000e-26,128.0,NaN
45,IG5:4344:4424,CP034058.1,95.062,81,4,0,1,81,3665087,3665167,9.470000e-26,128.0,NaN
46,IG5:4344:4424,CP000117.1,95.062,81,4,0,1,81,5173758,5173838,9.470000e-26,128.0,NaN
51,IG10:6205:6301,CP047242.1,98.969,97,1,0,1,97,110408,110504,1.570000e-39,174.0,NaN
52,IG10:6205:6301,CP034058.1,98.969,97,1,0,1,97,3666948,3667044,1.570000e-39,174.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,IG74:44856:45099,CP034058.1,95.492,244,11,0,1,244,2498666,2498909,4.760000e-104,390.0,NaN
607,IG74:44856:45099,CP000117.1,95.492,244,11,0,1,244,5214270,5214513,4.760000e-104,390.0,NaN
616,IG82:47415:47640,CP047242.1,95.133,226,11,0,1,226,151618,151843,4.410000e-94,357.0,NaN
617,IG82:47415:47640,CP034058.1,95.133,226,11,0,1,226,2501225,2501450,4.410000e-94,357.0,NaN


In [ ]:
# Create a record
recordn = SeqRecord(sequence_object,
                   id='fragement22', # random accession number
                   name='fragement22',
                   description='fragement22')
recordn.annotations = {'molecule_type': 'DNA'}
numbern = 0
sn = set()

In [ ]:
add_features_from_df(recordn, numbern, sn, uniprot_df)